# Pulumi (Infrastructure as Code with real languages)

## Goals
- Understand Pulumi’s core principles: **resources as code**, **stacks + state**, and **preview/update**.
- Know what Pulumi is used for, why teams adopt it, and what it’s capable of.
- See small **pseudo-code** examples of typical Pulumi programs and workflows.


## Prerequisites
- Basic cloud concepts: networking (VPC/subnets), IAM, regions/zones.
- Comfort in at least one Pulumi language (TypeScript, Python, Go, C#, Java).
- CLI comfort (running commands, reading diffs).


## What is Pulumi?

**Pulumi** is an Infrastructure-as-Code (IaC) platform where infrastructure is defined using **general-purpose programming languages** instead of a dedicated DSL.

You write a Pulumi program that declares cloud resources (AWS, Azure, GCP, Kubernetes, SaaS providers, etc.). Pulumi then computes the changes needed and performs an update by calling provider APIs.


## Core principles (mental model)
- **Infrastructure as real code**: use loops, functions, packages, tests, and type systems (depending on language).
- **Declarative outcome, imperative expression**: you write code, but Pulumi builds a **resource graph** (desired state) from it.
- **Preview → Update safety**: `pulumi preview` shows the diff before applying changes with `pulumi up`.
- **Stacks + state**: each environment (dev/stage/prod) is typically a **stack** with its own state.
- **Outputs**: many values are only known after creation (IDs, ARNs, endpoints). Pulumi models these as `Output` values.
- **Config + secrets**: runtime configuration is per-stack; secrets are encrypted and stored with the state.

A useful picture:
`program (code) + config + state  →  preview (diff)  →  up (API calls)  →  new state`


## What is Pulumi used for (practically)?

Pulumi is a strong fit when you want IaC, but you also want the power of a full programming language.

Common uses:
- Provision cloud infrastructure: networks, compute, storage, databases, IAM.
- Manage Kubernetes and platform tooling with code-level abstractions.
- Build higher-level building blocks ("components") that hide complexity and enforce standards.

Why teams use it:
- **Better abstractions**: loops/conditionals and shared libraries can reduce duplication.
- **Single-language workflow**: teams can use familiar tooling (linters, type checking, tests).
- **Policy + guardrails**: enforce org rules (e.g., “all buckets must have encryption”).
- **Multi-cloud + SaaS**: unify provisioning across many providers.


## Core concepts (minimum you should know)
- **Project**: a Pulumi program + metadata (like a repo/module).
- **Stack**: an instance of that project for an environment (dev/stage/prod), with its own config + state.
- **Resource**: a managed object (e.g., `aws.s3.Bucket`, `kubernetes.apps.v1.Deployment`).
- **Component**: a reusable abstraction composed of multiple resources.
- **Config**: per-stack settings; **secrets** are encrypted.
- **Outputs**: values resolved after deployment (IDs, ARNs, URLs), used to wire dependencies.


## Typical workflow
1) Create a project: `pulumi new`
2) Select or create a stack: `pulumi stack init` / `pulumi stack select`
3) Set config: `pulumi config set ...`
4) Preview changes: `pulumi preview`
5) Apply changes: `pulumi up`

State storage (backend) options include the Pulumi Service or self-managed storage (depending on your setup).


## Example: small AWS stack (pseudo-code)

Below is **illustrative** Pulumi Python-style code. It’s not meant to be copied blindly.

```python
# PSEUDO-CODE (Pulumi Python)

import pulumi
import pulumi_aws as aws

config = pulumi.Config()
env = pulumi.get_stack()

# 1) S3 bucket with opinionated defaults
bucket = aws.s3.Bucket(
    f'app-{env}',
    tags={'env': env},
    versioning=aws.s3.BucketVersioningArgs(enabled=True),
)

# 2) A tiny VPC example with a loop (language features are first-class)
vpc = aws.ec2.Vpc(
    'vpc',
    cidr_block='10.0.0.0/16',
    enable_dns_hostnames=True,
    tags={'env': env},
)

azs = aws.get_availability_zones(state='available').names[:2]
public_subnets = [
    aws.ec2.Subnet(
        f'public-{i}',
        vpc_id=vpc.id,
        cidr_block=f'10.0.{i}.0/24',
        availability_zone=az,
        tags={'env': env},
    )
    for i, az in enumerate(azs)
]

pulumi.export('bucket_name', bucket.bucket)
pulumi.export('vpc_id', vpc.id)
pulumi.export('public_subnet_ids', pulumi.Output.all(*[s.id for s in public_subnets]))
```

Typical commands:
```bash
pulumi new aws-python
pulumi stack init dev
pulumi config set aws:region us-east-1
pulumi preview
pulumi up
# pulumi destroy
```


## What Pulumi is capable of
- Build reusable abstractions (components) with strong defaults and guardrails.
- Use language features (loops, functions, packages) to generate consistent infrastructure.
- Manage many providers (cloud, Kubernetes, SaaS) in one program.
- Encrypt secrets in config/state and integrate with secret managers (depending on setup).
- Add policy checks and CI workflows around previews/updates.


## Pitfalls & quick tips
- Understand `Output` values: you often can’t use resource outputs like normal strings until they’re resolved.
- Keep secrets out of plain outputs/logs; assume state may contain sensitive metadata.
- Make resource naming deterministic and stable to avoid accidental replacements.
- Prefer small, well-tested components over copy-pasting patterns across projects.

## Exercises
- Create a component that provisions an S3 bucket with encryption + versioning + public access blocked.
- Add stack configs for `dev` and `prod` and show how they differ safely.
- Export outputs and wire them into a second component (e.g., app config).

## References
- Pulumi Docs: https://www.pulumi.com/docs/
- Pulumi AWS SDK: https://www.pulumi.com/registry/packages/aws/
